<a href="https://colab.research.google.com/github/JihunSKKU/PyMongo/blob/main/MongoDB_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyMongo basic

In [1]:
!pip install pymongo

In [2]:
import pymongo
from pymongo import MongoClient

In [15]:
CONNECTION_STRING = "~~~~~"
my_client = MongoClient(CONNECTION_STRING)

In [16]:
database = my_client['root']
database

Database(MongoClient(host=['jihunkim-shard-00-02.fdzvz.mongodb.net:27017', 'jihunkim-shard-00-01.fdzvz.mongodb.net:27017', 'jihunkim-shard-00-00.fdzvz.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='JihunKim', authsource='admin', replicaset='atlas-f3mmtn-shard-0', tls=True), 'root')

In [17]:
collection = database['products']
collection

Collection(Database(MongoClient(host=['jihunkim-shard-00-02.fdzvz.mongodb.net:27017', 'jihunkim-shard-00-01.fdzvz.mongodb.net:27017', 'jihunkim-shard-00-00.fdzvz.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='JihunKim', authsource='admin', replicaset='atlas-f3mmtn-shard-0', tls=True), 'root'), 'products')

## CRUD Operations

### Create Operations

- Create or insert operations add new documents to a collection.
    - collection.insert_one()
    - collection.insert_many()

In [18]:
import json

doc_list = []
for line in open('products.json', 'r'):
    doc_list.append(json.loads(line))

In [19]:
for doc in doc_list:
    try:
        collection.insert_one(doc)
    except pymongo.errors.DuplicateKeyError:
        pass
        # skip any documents with the same ID as the added document.

In [20]:
try:
    collection.insert_many(doc_list)
except pymongo.errors.BulkWriteError:
    pass
    # if there is a document with the same id as the added document
    # list, do not include the list itself.

In [21]:
results = collection.find()
print(type(results)) # Cursor has dictionaries.

for result in results:
    print(result)

<class 'pymongo.synchronous.cursor.Cursor'>
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 250, 'warranty_years': 1, 'available': True}
{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 0}
{'_id': 'dial8'

### Read Operations

Read operations retrieve documents from a collection; i.e.
query a collection for documents.
- collection.find()
- collection.find_one()


In [22]:
collection.find()

In [23]:
collection.find_one()

{'_id': 'dial1',
 'name': 'iPhone 14',
 'brand': 'Apple',
 'type': 'phone',
 'price': 250,
 'warranty_years': 1,
 'available': True}

### Update Operations

- Update operations modify existing documents in a collection.
    - collection.update_one()
    - collection.update_many()


- **What is a Filter?**
    - A query or a set of conditions you specify to retrieve documents from a collection that match those conditions.
    - Filters are used with various MongoDB operations, such as `find()`, `deleteMany()`, `updateMany()`, and `aggregate()`, to narrow down the set of documents that you want to work with or modify.

- **Example**

    ```Python
    collection.find({"color": "green"})
    ```
    - This query will return all documents in the collection where the key **"color"** equals the value **"green"**


- Modify the document in the collection
    - **$set, $unset**: Add/delete the value itself.
    - **$push, $pull**: Add/delete an element in a value that is in the form of a list.
- Change the value of a single document in the collection
    ```python
    collection.update_one({}, {"$set": {"key": "value"}})
    ```
- Change the values of the corresponding documents in the
collection
    ```python
    collection.update_many({}, {"$set": {"key": "value"}})
    ```
- Note: For the update operation, you must always provide an
empty filter!


Modify documents in the collection:

- **\$set**: Add/Modify the key and its value.
- **\$unset**: Remove the key and its value.
- **\$push**: Add an element to a list-shaped value.
- **\$pull**: Remove an element from a list-shaped value.

In [25]:
collection.update_one({}, {"$set": {"price": 200}})
collection.find_one()

{'_id': 'dial1',
 'name': 'iPhone 14',
 'brand': 'Apple',
 'type': 'phone',
 'price': 200,
 'warranty_years': 1,
 'available': True}

Since the original value is not in a list format, you cannot add the value using **\$push**. You should use **\$set**.

In [26]:
print("Before any operations")
print(collection.find_one())

# The code below generates an error
collection.update_one({}, {"$push": {"type": "mobile"}})

Before any operations
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': True}


WriteError: The field 'type' must be an array but is of type string in document {_id: "dial1"}, full error: {'index': 0, 'code': 2, 'errmsg': 'The field \'type\' must be an array but is of type string in document {_id: "dial1"}'}

In [27]:
print("Before any operations")
print(collection.find_one())

collection.update_one({}, {"$set": {"type": ["phone", "mobile"]}})
print("After $set operation")
print(collection.find_one())

collection.update_one({}, {"$pull": {"type": "mobile"}})
print("After $pull operation")
print(collection.find_one())

Before any operations
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': True}
After $set operation
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': ['phone', 'mobile'], 'price': 200, 'warranty_years': 1, 'available': True}
After $pull operation
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': ['phone'], 'price': 200, 'warranty_years': 1, 'available': True}


### Delete Operations
- Delete operations remove documents from a collection.
    - collection.delete_one()
    - collection.delete_many()


In [28]:
collection.delete_one({})
for product in collection.find():
    print(product)

{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 0}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 0.25, 'available': False, 'for': ['dial1', 'dial2']}
{'_i

### Sort operation

In [29]:
for product in collection.find().sort("price", 1):
    print(product)

{'_id': 'dial9', 'name': 'iPhone 14 Mini Case Yellow', 'type': ['accessory', 'case'], 'color': 'yellow', 'price': 10, 'warranty_years': 0}
{'_id': 'dial10', 'name': 'Galaxy S23 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 10, 'warranty_years': 0}
{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 0}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 0.25, 'available': False, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 0.25, 'available': True, 'for': 'dial4'}
{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warrant

### Count operation

In [30]:
num_all = collection.count_documents({})
num_case = collection.count_documents({"type": "case"})
print("# of products:", num_all)
print("# of cases:", num_case)

# of products: 10
# of cases: 5


## CRUD with Filtering

In [33]:
for product in collection.find({"type": "case"}):
    print(product)

{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 0}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 0.25, 'available': False, 'for': ['dial1', 'dial2']}
{'_id': 'dial9', 'name': 'iPhone 14 Mini Case Yellow', 'type': ['accessory', 'case'], 'color': 'yellow', 'price': 10, 'warranty_years': 0}
{'_id': 'dial10', 'name': 'Galaxy S23 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 10, 'warranty_years': 0}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 0.25, 'available': True, 'for': 'dial4'}


In [34]:
for product in collection.find({"price": {"$gt": 100}}):
    print(product)

{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}


In [35]:
collection.update_many({"type": "case"}, {"$set": {"warranty_years": 1}})

for product in collection.find({"type": "case"}):
    print(product)

{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 1}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 1, 'available': False, 'for': ['dial1', 'dial2']}
{'_id': 'dial9', 'name': 'iPhone 14 Mini Case Yellow', 'type': ['accessory', 'case'], 'color': 'yellow', 'price': 10, 'warranty_years': 1}
{'_id': 'dial10', 'name': 'Galaxy S23 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 10, 'warranty_years': 1}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 1, 'available': True, 'for': 'dial4'}


In [40]:
collection.update_one({"_id": "dial8"}, {"$pull": {"for": "dial2"}})

for product in collection.find():
    print(product)

{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 1}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 1, 'available': False, 'for': ['dial1']}
{'_id': 'dial9',

In [41]:
collection.delete_one({"name": "Galaxy S23 Case Green"})

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000000ad'), 'opTime': {'ts': Timestamp(1730362591, 2), 't': 173}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1730362591, 2), 'signature': {'hash': b'[J\xaa\xc8\x16\xf7T\x82\t\xcda\xde\xd7\x0b\xd3\xba\xa8\xdc\xb6M', 'keyId': 7384356045314850820}}, 'operationTime': Timestamp(1730362591, 2)}, acknowledged=True)

In [42]:
collection.delete_many({"brand": "Apple"})

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000000ad'), 'opTime': {'ts': Timestamp(1730362603, 6), 't': 173}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1730362603, 6), 'signature': {'hash': b"\xee'\xba/rL8\x95\xcbl\xd5ZT\xc5\xbb\xb1\x10 \xbc\xa8", 'keyId': 7384356045314850820}}, 'operationTime': Timestamp(1730362603, 6)}, acknowledged=True)

In [43]:
for product in collection.find({"price": {"$lte": 100}}).sort("price", -1):
    print(product)

{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 1, 'available': True, 'for': 'dial4'}
{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 1}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 1, 'available': False, 'for': ['dial1']}
{'_id': 'dial9', 'name': 'iPhone 14 Mini Case Yellow', 'type': ['accessory', 'case'], 'color': 'yellow', 'price': 10, 'warranty_years': 1}


In [44]:
collection.update_many({"type": "phone"}, {"$set": {"warranty_years": 2}})

results = collection.find({"type": "phone"})
for result in results:
    print(result)

{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 2, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 2, 'available': False}


In [45]:
collection.update_many({"type": "charger"}, {"$mul": {"price": 1.2}})
for product in collection.find({"type": "charger"}):
    print(product)

{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 30.0, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 18.0, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}


In [46]:
collection.delete_many({"type": "case"})
for product in collection.find({"type": "case"}):
    print(product)

In [47]:
collection.delete_many({"$and": [{"color": "green"}, {"price": {"$lte": 12}}]})
for product in collection.find({"$and": [{"color": "green"}, {"price": {"$lte": 12}}]}):
    print(product)